In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import cv2
import numpy as np
from tqdm import tqdm

In [5]:
DATADIR = '/content/drive/Shareddrives/AAML_final/Training_Dataset'
CATEGORIES = ['Fire', 'NoFire']

In [6]:
IMG_SIZE = 64
def create_training_data():
    training_data = []
    for category in CATEGORIES:  

        path = os.path.join(DATADIR,category) 
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=C 1=O

        for img in tqdm(os.listdir(path)):  # iterate over each image
            try:
                img_array = cv2.imread(os.path.join(path,img))  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass
              
    return training_data

In [7]:
training_data = create_training_data()

  1%|          | 11/1124 [00:36<1:01:41,  3.33s/it]


KeyboardInterrupt: ignored

In [ ]:
import random

print(len(training_data))
random.shuffle(training_data)
for sample in training_data[:10]:
    print(sample[1])

In [ ]:
X = []
Y = []

for features,label in training_data:
    X.append(features)
    Y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
X = X/255.0
Y = np.array(Y)
X.shape[1:]

In [ ]:
# # set up image augmentation
# from keras.preprocessing.image import ImageDataGenerator

# datagen = ImageDataGenerator(
#     rotation_range=15,
#     horizontal_flip=True,
#     width_shift_range=0.1,
#     height_shift_range=0.1
#     #zoom_range=0.3
#     )
# datagen.fit(X)

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, AveragePooling2D

model = Sequential()


model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=X.shape[1:]))
model.add(AveragePooling2D())
model.add(Dropout(0.5))

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D())
model.add(Dropout(0.5))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D())
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(units=128, activation='relu'))

model.add(Dense(units=2, activation = 'softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X, Y, batch_size=32, epochs=100, validation_split=0.3)
# model.fit_generator(datagen.flow(X, Y, batch_size=32),
#                     epochs=100,
#                     verbose=1)

In [ ]:
model.save('/content/drive/Shareddrives/AAML_final/FireNet-LightWeight-Network-for-Fire-Detection/Trained_Model_FIreNet/train1.h5')

In [ ]:
from matplotlib import pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [14]:
import tensorflow as tf
from tensorflow.keras.utils import plot_model
model = tf.keras.models.load_model('/content/drive/Shareddrives/AAML_final/FireNet-LightWeight-Network-for-Fire-Detection/Trained_Model_FIreNet/train1.h5')

# model.fit_generator(datagen.flow(X, Y, batch_size=32),
#                     epochs=100,
#                   verbose=1)


In [ ]:
 plot_model(model, to_file='model_small.png', show_layer_names=False, show_shapes=True)

#h5 to tflite

In [18]:
import tensorflow as tf
model = tf.keras.models.load_model('/content/drive/Shareddrives/AAML_final/FireNet-LightWeight-Network-for-Fire-Detection/Trained_Model_FIreNet/Fire-64x64-color-v7-soft.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tf_model = converter.convert()
open('/content/drive/Shareddrives/AAML_final/FireNet-LightWeight-Network-for-Fire-Detection/Trained_Model_FIreNet/Fire-64x64-color-v7-soft.tflite', 'wb').write(tf_model)

INFO:tensorflow:Assets written to: /tmp/tmp_hq55en2/assets


INFO:tensorflow:Assets written to: /tmp/tmp_hq55en2/assets


2591340